<a href="https://colab.research.google.com/github/RohithRaparthi/FMML_LABS/blob/main/face_mask_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [3]:

# Step 1: Mount Google Drive and Load Dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

# Dataset Path (Modify as per your dataset)
dataset_dir = '/content/drive/MyDrive/archive/'


In [10]:

# Step 2: Data Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = data_gen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = data_gen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 610 images belonging to 2 classes.
Found 152 images belonging to 2 classes.


In [16]:

# Step 3: Build the Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
# Changed 'lr' to 'learning_rate' in Adam optimizer


In [17]:
# Step 4: Train the Model
epochs = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 278s 14s/step - accuracy: 0.9168 - loss: 0.2441 - val_accuracy: 1.0000 - val_loss: 0.0063
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 1.0000 - loss: 0.0042 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 7.2021e-04
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 1.0000 - loss: 9.9153e-04 - val_accuracy: 1.0000 - val_loss: 6.9773e-04
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 1.0000 - loss: 7.2865e-04 - val_accuracy: 1.0000 - val_loss: 6.0514e-04
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 1.0000 - loss: 6.5086e-04 - val_accuracy: 1.0000 - val_loss: 5.9455e-04
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 1.0000 - loss: 5.1142e-04 

In [13]:
# Save Model
model.save('/content/mask_detector_model.h5')


In [14]:
# Step 5: Real-Time Face Mask Detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
model = load_model('/content/mask_detector_model.h5')

def detect_mask(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face = np.expand_dims(face, axis=0)
        face = preprocess_input(face)
        mask_prediction = model.predict(face)[0][0]
        label = "Mask" if mask_prediction > 0.5 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    return frame

In [18]:

# Open Webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = detect_mask(frame)
    cv2.imshow('Face Mask Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()